<a href="https://colab.research.google.com/github/carighi/al_ml_workshop/blob/main/Feature_Scaling_Numerical_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Scale Numerical Data**

Many machine learning algorithms perform better when numerical input variables are scaled
to a standard range. This includes algorithms that use a weighted sum of the input, like
linear regression, and algorithms that use distance measures, like k-nearest neighbors. The two
most popular techniques for scaling numerical data prior to modeling are normalization and
standardization.
* **Normalization** scales each input variable separately to the range of 0-1, which is
the range for
floating-point values where we have the most precision.
* **Standardization** scales
each input variable separately by subtracting the mean (called centering) and dividing by the
standard deviation to shift the distribution to have a mean of zero and a standard deviation of
one.

In this tutorial, you will learn:

* Data scaling is a recommended pre-processing step when working with many machine
learning algorithms.
* Data scaling can be achieved by normalizing or standardizing real-valued input and output
variables.
* How to apply standardization and normalization to improve the performance of predictive
modeling algorithms.

Adapted from Jason Brownlee. 2020. [Data Preparation for Machine Learning](https://machinelearningmastery.com/data-preparation-for-machine-learning/).

#Numerical Data Scaling Methods



##Data Normalization
Normalization is a rescaling of the data from the original range so that all values are within the
new range of 0 and 1. Normalization requires that you know or are able to accurately estimate
the minimum and maximum observable values. You may be able to estimate these values from
your available data.

In [ ]:
# example of a normalization
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
# define data
data = asarray([[100, 0.001],
[8, 0.05],
[50, 0.005],
[88, 0.07],
[4, 0.1]])
print(data)
# define min max scaler
# Transform features by scaling each feature to a given range.
# This estimator scales and translates each feature individually such
# that it is in the given range on the training set, e.g. between
# zero and one.
scaler = MinMaxScaler()
# Fit to data, then transform it.
scaled = scaler.fit_transform(data)
print(scaled)

##Data Standardization
Standardizing a dataset involves rescaling the distribution of values so that the mean of observed
values is 0 and the standard deviation is 1. This can be thought of as subtracting the mean
value or centering the data. Like normalization, standardization can be useful, and even
required in some machine learning algorithms when your data has input values with differing
scales. Standardization assumes that your observations fit a Gaussian distribution (bell curve)
with a well-behaved mean and standard deviation. You can still standardize your data if this
expectation is not met, but you may not get reliable results.

In [ ]:
# example of a standardization
from numpy import asarray
from sklearn.preprocessing import StandardScaler
# define data
data = asarray([[100, 0.001],
[8, 0.05],
[50, 0.005],
[88, 0.07],
[4, 0.1]])
print(data)
# define standard scaler
# Standardize features by removing the mean and scaling to unit variance.
scaler = StandardScaler()
# Fit to data, then transform it.
scaled = scaler.fit_transform(data)
print(scaled)

#Diabetes Dataset
The dataset classifies patient data as
either an onset of diabetes within five years or not. There are 768 examples and eight input variables. It is a binary classification problem.

You can learn more about the dataset here:

* Diabetes Dataset File ([pima-indians-diabetes.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv))
* Diabetes Dataset Details ([pima-indians-diabetes.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names))

###Download Diabetes data files

In [ ]:
!wget "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv" -O pima-indians-diabetes.csv
!wget "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names" -O pima-indians-diabetes.names
!head pima-indians-diabetes.csv

In [ ]:
# load and summarize the diabetes dataset
from pandas import read_csv
from matplotlib import pyplot
# load the dataset
dataset = read_csv('pima-indians-diabetes.csv', header=None)
# summarize the shape of the dataset
print(dataset.shape)
# summarize each variable
print(dataset.describe())

This confirms the 8
input variables, one output variable, and 768 rows of data. A statistical summary of the input
variables is provided show that each variable has a very different scale. This makes it a good
dataset for exploring data scaling methods.

We can create a histogram for each input variable. The plots confirm the differing scale
for each input variable and show that the variables have different scales.


In [ ]:
# histograms of the variables
fig = dataset.hist(xlabelsize=4, ylabelsize=4)
[x.title.set_size(4) for x in fig.ravel()]
# show the plot
pyplot.show()

Next, let's fit and evaluate a machine learning model on the raw dataset. We will use
a k-nearest neighbor algorithm with default hyperparameters and evaluate it using repeated
stratified k-fold cross-validation.

In [ ]:
# evaluate knn on the raw diabetes dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
# load the dataset
dataset = read_csv('pima-indians-diabetes.csv', header=None)
data = dataset.values
# separate into input and output columns
X, y = data[:, :-1], data[:, -1]
# ensure inputs are floats and output is an integer label
X = X.astype('float32')
y = LabelEncoder().fit_transform(y.astype('str'))
# define and configure the model
model = KNeighborsClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report model performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In this case we can see that the model achieved a mean classification accuracy of about
71.7 percent.

#MinMaxScaler Transform
We can apply the **MinMaxScaler** to the diabetes dataset directly to normalize the input variables.
We will use the default configuration and scale values to the range 0 and 1. First, a **MinMaxScaler**
instance is defined with default hyperparameters. Once defined, we can call the **fit.transform**()
function and pass it to our dataset to create a transformed version of our dataset.

##Summary of each input variable

In [ ]:
# visualize a minmax scaler transform of the diabetes dataset
from pandas import read_csv
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
# load the dataset
dataset = read_csv('pima-indians-diabetes.csv', header=None)
# retrieve just the numeric input values
data = dataset.values[:, :-1]
# perform a min-max scaler transform of the dataset
trans = MinMaxScaler()
data = trans.fit_transform(data)
# convert the array back to a dataframe
dataset = DataFrame(data)
# summarize
print(dataset.describe())

We can see that the
distributions have been adjusted and that the minimum and maximum values for each variable
are now 0.0 and 1.0 respectively.

##Histogram plots of the variables

In [ ]:
# histograms of the variables
fig = dataset.hist(xlabelsize=4, ylabelsize=4)
[x.title.set_size(4) for x in fig.ravel()]
# show the plot
pyplot.show()

Histogram plots of the variables are created, although the distributions don't look much
different from their original distributions seen in the previous section. We can confirm that the
minimum and maximum values are zero and one respectively, as we expected.

##Model evaluation

Next, let's evaluate the same KNN model as the previous section, but in this case, on a
MinMaxScaler transform of the dataset.

In [ ]:
# evaluate knn on the diabetes dataset with minmax scaler transform
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
# load the dataset
dataset = read_csv('pima-indians-diabetes.csv', header=None)
data = dataset.values
# separate into input and output columns
X, y = data[:, :-1], data[:, -1]
# ensure inputs are floats and output is an integer label
X = X.astype('float32')
y = LabelEncoder().fit_transform(y.astype('str'))
# define the pipeline
trans = MinMaxScaler()
model = KNeighborsClassifier()
pipeline = Pipeline(steps=[('t', trans), ('m', model)])
# evaluate the pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report pipeline performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

We can see that the MinMaxScaler transform results in a lift in
performance from 71.7 percent accuracy without the transform to about 73.9 percent with the
transform.

#StandardScaler Transform
We can apply the StandardScaler to the diabetes dataset directly to standardize the input
variables. We will use the default configuration and scale values to subtract the mean to center
them on 0.0 and divide by the standard deviation to give the standard deviation of 1.0. First, a
StandardScaler instance is defined with default hyperparameters. Once defined, we can call
the fit transform() function and pass it to our dataset to create a transformed version of our
dataset.

##Summary of each input variable

In [ ]:
# visualize a standard scaler transform of the diabetes dataset
from pandas import read_csv
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot
# load the dataset
dataset = read_csv('pima-indians-diabetes.csv', header=None)
# retrieve just the numeric input values
data = dataset.values[:, :-1]
# perform a robust scaler transform of the dataset
trans = StandardScaler()
data = trans.fit_transform(data)
# convert the array back to a dataframe
dataset = DataFrame(data)
# summarize
print(dataset.describe())

We can see that the
distributions have been adjusted and that the mean is a very small number close to zero and
the standard deviation is very close to 1.0 for each variable.

## Histogram plots of the variables

In [ ]:
# histograms of the variables
fig = dataset.hist(xlabelsize=4, ylabelsize=4)
[x.title.set_size(4) for x in fig.ravel()]
# show the plot
pyplot.show()

Histogram plots of the variables are created, although the distributions don't look much
different from their original distributions seen in the previous section other than their scale on
the x-axis. We can see that the center of mass for each distribution is centered on zero, which is
more obvious for some variables than others.

##Model evaluation
Next, let's evaluate the same KNN model as the previous section, but in this case, on a
StandardScaler transform of the dataset. The complete example is listed below.

In [ ]:
# evaluate knn on the diabetes dataset with standard scaler transform
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# load the dataset
dataset = read_csv('pima-indians-diabetes.csv', header=None)
data = dataset.values
# separate into input and output columns
X, y = data[:, :-1], data[:, -1]
# ensure inputs are floats and output is an integer label
X = X.astype('float32')
y = LabelEncoder().fit_transform(y.astype('str'))
# define the pipeline
trans = StandardScaler()
model = KNeighborsClassifier()
pipeline = Pipeline(steps=[('t', trans), ('m', model)])
# evaluate the pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report pipeline performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

We can see that the StandardScaler transform results in a lift in
performance from 71.7 percent accuracy without the transform to about 74.1 percent with the
transform, slightly higher than the result using the MinMaxScaler that achieved 73.9 percent.